In [1]:
from fastai.collab import *
from fastai.tabular.all import *

In [2]:
path = untar_data(URLs.ML_100k)
path

Path('/Users/imad/.fastai/data/ml-100k')

In [5]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None, names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
movies = pd.read_csv(path/'u.item', delimiter='|', header=None, names=['movie', 'title'], encoding='latin-1', usecols=[0, 1])
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [10]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [13]:
ratings.shape

(100000, 5)

In [19]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.classes

{'user': (#944) ['#na#',1,2,3,4,5,6,7,8,9...],
 'title': (#1665) ['#na#',"'Til There Was You (1997)",'1-900 (1994)','101 Dalmatians (1996)','12 Angry Men (1957)','187 (1997)','2 Days in the Valley (1996)','20,000 Leagues Under the Sea (1954)','2001: A Space Odyssey (1968)','3 Ninjas: High Noon At Mega Mountain (1998)'...]}

In [36]:
dls.show_batch()

,user,title,rating
0,271,E.T. the Extra-Terrestrial (1982),4
1,7,"Wizard of Oz, The (1939)",5
2,817,Heat (1995),5
3,405,High Noon (1952),3
4,90,All About Eve (1950),5
5,507,"Phantom, The (1996)",5
6,727,"Silence of the Lambs, The (1991)",4
7,671,Executive Decision (1996),5
8,346,Carlito's Way (1993),4
9,308,My Left Foot (1989),4


In [20]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_users, n_movies

(944, 1665)

In [21]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors):
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.movie_factors = nn.Embedding(n_movies, n_factors)
    
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return (users * movies).sum(dim=1)

In [25]:
dls.one_batch()[0].shape, dls.one_batch()[1].shape

(torch.Size([64, 2]), torch.Size([64, 1]))

In [27]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,48.491528,46.122906,00:10
1,22.627960,24.253294,00:06
2,7.020413,11.650857,00:06
3,3.576531,8.371878,00:06
4,2.502200,7.973752,00:06


In [28]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(.5, 5)):
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.movie_factors = nn.Embedding(n_movies, n_factors)
        self.y_range = y_range
    
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        return sigmoid_range((users * movies).sum(dim=1), *self.y_range)

In [29]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,5.652528,5.837720,00:09
1,5.110436,5.622582,00:11
2,4.080307,5.377186,00:10
3,3.631600,5.223083,00:14
4,3.092148,5.196243,00:14


In [32]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(.5, 5)):
        self.user_factors = nn.Embedding(n_users, n_factors)
        self.user_bias = nn.Embedding(n_users, 1)
        self.movie_factors = nn.Embedding(n_movies, n_factors)
        self.movie_bias = nn.Embedding(n_movies, 1)        
        self.y_range = y_range
    
    def forward(self, x):
        users = self.user_factors(x[:, 0])
        movies = self.movie_factors(x[:, 1])
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias(x[:, 0]) + self.movie_bias(x[:, 1])
        return sigmoid_range(res, *self.y_range)

In [33]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,5.573811,5.580082,00:11
1,4.602935,4.878285,00:07
2,3.598750,4.366551,00:07
3,2.845090,4.148434,00:07
4,2.516217,4.114805,00:07


In [44]:
def create_params(size):
    return nn.Parameter(torch.zeros(*size).normal_(0, .01))

In [45]:
class DotProduct(Module):
    def __init__(self, n_users, n_movies, n_factors, y_range=(.5, 5)):
        self.user_factors = create_params([n_users, n_factors])
        self.user_bias = create_params([n_users, 1])
        self.movie_factors = create_params([n_movies, n_factors])
        self.movie_bias = create_params([n_movies, 1]) 
        self.y_range = y_range
    
    def forward(self, x):
        users = self.user_factors[x[:, 0]]
        movies = self.movie_factors[x[:, 1]]
        res = (users * movies).sum(dim=1, keepdim=True)
        res += self.user_bias[x[:, 0]] + self.movie_bias[x[:, 1]]
        return sigmoid_range(res, *self.y_range)

In [46]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.912734,0.924706,00:09
1,0.814983,0.848361,00:06
2,0.628584,0.836272,00:06
3,0.461301,0.850549,00:06
4,0.354000,0.854552,00:07


In [51]:
movie_bias = learn.model.movie_bias.squeeze()
[dls.classes['title'][i] for i in movie_bias.argsort()[:5]]

['Children of the Corn: The Gathering (1996)',
 'Lawnmower Man 2: Beyond Cyberspace (1996)',
 'Cable Guy, The (1996)',
 "Joe's Apartment (1996)",
 'Tales from the Crypt Presents: Bordello of Blood (1996)']

In [52]:
movie_bias = learn.model.movie_bias.squeeze()
[dls.classes['title'][i] for i in movie_bias.argsort()[-5:]]

['Rear Window (1954)',
 'L.A. Confidential (1997)',
 "Schindler's List (1993)",
 'Shawshank Redemption, The (1994)',
 'Titanic (1997)']

In [53]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(5, 5e-5)

epoch,train_loss,valid_loss,time
0,1.861074,1.847219,00:11
1,1.790623,1.791885,00:07
2,1.754227,1.749619,00:07
3,1.721121,1.729035,00:07
4,1.731620,1.725625,00:07


In [54]:
class CollabNN(Module):
    def __init__(self, user_sz, item_sz, y_range=(0, 5.5), n_act=100):
        self.user_factors = nn.Embedding(*user_sz)
        self.movie_factors = nn.Embedding(*item_sz)
        self.layers = nn.Sequential(
            nn.Linear(user_sz[1] + item_sz[1], n_act),
            nn.ReLU(),
            nn.Linear(n_act, 1)
        )
        self.y_range = y_range
    
    def forward(self, x):
        embeddings = torch.cat([self.user_factors(x[:, 0]), self.movie_factors(x[:, 1])], dim=1)
        return sigmoid_range(self.layers(embeddings), *self.y_range)
        

In [56]:
model = CollabNN((n_users, 50), (n_movies, 50))
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,1.018861,0.998217,00:12
1,0.897466,0.928229,00:07
2,0.868444,0.895949,00:08
3,0.815504,0.877247,00:10
4,0.776726,0.881833,00:17


In [57]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5), use_nn=True, layers=[100, 50])
learn.fit_one_cycle(5, 5e-5)

epoch,train_loss,valid_loss,time
0,1.066979,1.038792,00:14
1,0.897078,0.906277,00:14
2,0.858467,0.892624,00:13
3,0.758879,0.888168,00:12
4,0.748551,0.889692,00:12


In [58]:
learn.model

EmbeddingNN(
  (embeds): ModuleList(
    (0): Embedding(944, 74)
    (1): Embedding(1665, 102)
  )
  (emb_drop): Dropout(p=0.0, inplace=False)
  (bn_cont): BatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): Sequential(
    (0): LinBnDrop(
      (0): BatchNorm1d(176, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=176, out_features=100, bias=False)
      (2): ReLU(inplace=True)
    )
    (1): LinBnDrop(
      (0): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Linear(in_features=100, out_features=50, bias=False)
      (2): ReLU(inplace=True)
    )
    (2): LinBnDrop(
      (0): Linear(in_features=50, out_features=1, bias=True)
    )
    (3): SigmoidRange(low=0, high=5.5)
  )
)